![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings_JSL/Finance/8.Data_Augmentation_with_ChunkMappers.ipynb)

# Installation

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs 

In [ ]:
from johnsnowlabs import nlp, finance, viz
nlp.install(force_browser=True)

# Start Spark Session

In [4]:
spark = nlp.start()

👌 Launched cpu optimized session with with: 🚀Spark-NLP==4.2.4, 💊Spark-Healthcare==4.2.4, running on ⚡ PySpark==3.1.2


# Financial Data Augmentation with Chunk Mappers

# About Data Augmentation

__Data Augmentation__ is the process of increase an extracted datapoint with external sources. 

For example, let's suppose I work with a document which mentions the company _Amazon_. We could be talking about stock prices, or some legal litigations, or just a commercial agreement with a provider, among others.

In the document, we can extract a company name using NER as an Organization, but that's all the information available about the company in that document.

Well, with __Data Augmentation__, we can use external sources, as _SEC Edgar, Crunchbase, Nasdaq_ or even _Wikipedia_, to enrich the company with much more information, allowing us to take better decisions.

Let's see how to do it.

## Sample Texts from Cadence Design System

Examples taken from publicly available information about Cadence in SEC's Edgar database [here](https://www.sec.gov/Archives/edgar/data/813672/000081367222000012/cdns-20220101.htm) and [Wikipedia](https://en.wikipedia.org/wiki/Cadence_Design_Systems)

In [ ]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings_JSL/Finance/data/cdns-20220101.html.txt

In [6]:
with open('cdns-20220101.html.txt', 'r') as f:
  cadence_sec10k = f.read()
print(cadence_sec10k[:300])

Table of Contents
UNITED STATES SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
_____________________________________ 
FORM 10-K 
_____________________________________  
(Mark One)
☒
ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the fiscal year en


In [7]:
pages = [x for x in cadence_sec10k.split("Table of Contents") if x.strip() != '']
print(pages[0])


UNITED STATES SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
_____________________________________ 
FORM 10-K 
_____________________________________  
(Mark One)
☒
ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the fiscal year ended January 1, 2022 
OR
☐
TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the transition period from _________ to_________.

Commission file number 000-15867 
_____________________________________
 
CADENCE DESIGN SYSTEMS, INC. 
(Exact name of registrant as specified in its charter)
____________________________________ 
Delaware
 
00-0000000
(State or Other Jurisdiction ofIncorporation or Organization)
 
(I.R.S. EmployerIdentification No.)
2655 Seely Avenue, Building 5,
San Jose,
California
 
95134
(Address of Principal Executive Offices)
 
(Zip Code)
(408)
-943-1234 
(Registrant’s Telephone Number, including Area Code) 
Securities registered pursuant to Section 1

## Step 1: Using Text Classification to find Relevant Parts of the Document: 10K Summary
In this case, we know page 0 is always the page with summary information about the company. However, let's suppose we don't know it. We can use Page Classification.

To check the SEC 10K Summary page, we have a specific model called `"finclf_form_10k_summary_item"`

In [8]:
# Text Classifier
# This pipeline allows you to use different classification models to understand if an input text is of a specific class or is something else.
  
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

use_embeddings = nlp.UniversalSentenceEncoder.pretrained()\
    .setInputCols("document") \
    .setOutputCol("sentence_embeddings")

classifier = finance.ClassifierDLModel.pretrained("finclf_form_10k_summary_item", "en", "finance/models")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("category")

nlpPipeline = nlp.Pipeline(stages=[
    document_assembler, 
    use_embeddings,
    classifier])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
finclf_form_10k_summary_item download started this may take some time.
[OK!]


In [9]:
df = spark.createDataFrame([[pages[0]]]).toDF("text")

result = model.transform(df).cache()


In [10]:
result.select('category.result').show()

+------------------+
|            result|
+------------------+
|[form_10k_summary]|
+------------------+



##  Step 2: Named Entity Recognition on 10K Summary
Main component to carry out information extraction and extract entities from texts. 

This time we will use a model trained to extract many entities from 10K summaries.

In [11]:
sentenceDetector = nlp.SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings = nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

ner_model = finance.NerModel.pretrained("finner_sec_10k_summary", "en", "finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")

ner_converter = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

nlpPipeline = nlp.Pipeline(stages=[
    document_assembler,
    sentenceDetector,
    tokenizer,
    embeddings,
    ner_model,
    ner_converter,
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

light_model = nlp.LightPipeline(model)

bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[OK!]
finner_sec_10k_summary download started this may take some time.
[OK!]


## We use LightPipeline to get the result

In [12]:
import pandas as pd

ner_result = light_model.fullAnnotate(pages[0])

chunks = []
entities = []
begin = []
end = []

for n in ner_result[0]['ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    
df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 'entities':entities})

df.head(20)

,chunks,begin,end,entities
0,"January 1, 2022",287,301,FISCAL_YEAR
1,000-15867,476,484,CFN
2,"CADENCE DESIGN SYSTEMS, INC",527,553,ORG
3,Delaware,650,657,STATE
4,00-0000000,661,670,IRS
5,"2655 Seely Avenue, Building 5,\nSan Jose,\nCal...",772,822,ADDRESS
6,(408)\n-943-1234,886,900,PHONE
7,Common Stock,1098,1109,TITLE_CLASS
8,$0.01,1112,1116,TITLE_CLASS_VALUE
9,CDNS,1138,1141,TICKER


Alright! CADENCE DESIGN SYSTEMS, INC has been detected as an organization. 

Now, let's augment `CADENCE DESIGN SYSTEMS, INC` with more information about the company, given that there are no more details in the SEC10K form I can use.

But before __augmenting__, there is a very important step we need to carry out: `Company Name Normalization`

## Step 3: Company Names Normalization

Let's suppose we want to manually get information about CADENCE DESIGN SYSTEMS, INC.

Since it's a public US company, we can go to [SEC Edgar's database](https://www.sec.gov/edgar/searchedgar/companysearch) and look for it.


Unfortunately, `CADENCE DESIGN SYSTEMS, INC` is not the official name of the company, which means no entry for `CADENCE DESIGN SYSTEMS, INC` is available. That's were __Company Names Normalization__ comes in handy.

**Company Name Normalization** is the process of obtaining the name of the company used by data providers, usually the "official" name of the company.

Sometimes, some data providers may have different versions of the name with different punctuation. For example, for Meta:
- Meta Platforms, Inc.
- Meta Platforms Inc.
- Meta Platforms, Inc
- etc

So, it's mandatory we do `Company Normalization` taking into account the database / datasource provider we want to extract data from. The data providers we have are:
- SEC Edgar
- Crunchbase until 2015
- Wikidata (in progress)

Let's normalize `CADENCE DESIGN SYSTEMS, INC` to the official name in _SEC Edgar_.

In [13]:
resolver = finance.SentenceEntityResolverModel.pretrained("finel_edgar_company_name", "en", "finance/models")\
      .setInputCols(["sentence_embeddings"]) \
      .setOutputCol("resolution")\
      .setDistanceFunction("EUCLIDEAN")

pipelineModel = nlp.PipelineModel(
      stages = [
          document_assembler,
          use_embeddings,
          resolver])

lp_res = nlp.LightPipeline(pipelineModel)

finel_edgar_company_name download started this may take some time.
[OK!]


In [14]:
ORG = list(df[df["entities"] == "ORG"]["chunks"])

ORG

['CADENCE DESIGN SYSTEMS, INC', 'Cadence Design Systems, Inc']

In [15]:
el_res = lp_res.annotate(ORG)
el_res

[{'document': ['CADENCE DESIGN SYSTEMS, INC'],
  'sentence_embeddings': ['CADENCE DESIGN SYSTEMS, INC'],
  'resolution': ['CADENCE DESIGN SYSTEMS INC']},
 {'document': ['Cadence Design Systems, Inc'],
  'sentence_embeddings': ['Cadence Design Systems, Inc'],
  'resolution': ['CADENCE DESIGN SYSTEMS INC']}]

In [16]:
NORM_ORG = el_res[0]['resolution'][0]

NORM_ORG

'CADENCE DESIGN SYSTEMS INC'

Here is our normalized name for Amazon: `CADENCE DESIGN SYSTEMS INC`.

Now, let's see which information is available in Edgar database for `CADENCE DESIGN SYSTEMS INC`

Once we have the normalized name of the company, we can use `John Snow Labs Chunk Mappers`. These are pretrained data sources, which are updated frequently and can be queried inside Spark NLP without sending any API call to any server.

In this case, we will use Edgar Database (`finmapper_edgar_companyname`)




## Step 4: Data Augmentation with Chunk Mappers


Once we have the normalized name of the company, we can use `John Snow Labs Chunk Mappers`. These are pretrained data sources, which are updated frequently and can be queried inside Spark NLP without sending any API call to any server.

In this case, we will use Edgar Database (`finmapper_edgar_companyname`)

The component which carries out __Data Augmentation__ is called `ChunkMapper`.

It's name comes from the way it works: it uses a _Ner Chunk_ to map it to an external data source.

As a result, you will get a JSON with a dictionary of additional fields and their values. 

Let's take a look at how it works.

In [17]:
chunk_assembler = nlp.Doc2Chunk()\
    .setInputCols("document") \
    .setOutputCol("chunk") \
    .setIsArray(False)

CM = finance.ChunkMapperModel().pretrained("finmapper_edgar_companyname", "en", "finance/models")\
    .setInputCols(["chunk"])\
    .setOutputCol("mappings")

cm_pipeline = nlp.Pipeline(stages=[document_assembler, chunk_assembler, CM])

fit_cm_pipeline = cm_pipeline.fit(empty_data)

finmapper_edgar_companyname download started this may take some time.
[OK!]


In [18]:
# LightPipelines don't support Doc2Chunk, so we will use here usual transform
df = spark.createDataFrame([[NORM_ORG]]).toDF("text")

df.show(truncate = False)

+--------------------------+
|text                      |
+--------------------------+
|CADENCE DESIGN SYSTEMS INC|
+--------------------------+



In [19]:
res = fit_cm_pipeline.transform(df)
res.show()

+--------------------+--------------------+--------------------+--------------------+
|                text|            document|               chunk|            mappings|
+--------------------+--------------------+--------------------+--------------------+
|CADENCE DESIGN SY...|[{document, 0, 25...|[{chunk, 0, 25, C...|[{labeled_depende...|
+--------------------+--------------------+--------------------+--------------------+



In [20]:
r = res.collect()
r

[Row(text='CADENCE DESIGN SYSTEMS INC', document=[Row(annotatorType='document', begin=0, end=25, result='CADENCE DESIGN SYSTEMS INC', metadata={'sentence': '0'}, embeddings=[])], chunk=[Row(annotatorType='chunk', begin=0, end=25, result='CADENCE DESIGN SYSTEMS INC', metadata={'sentence': '0', 'chunk': '0'}, embeddings=[])], mappings=[Row(annotatorType='labeled_dependency', begin=0, end=25, result='CADENCE DESIGN SYSTEMS INC', metadata={'sentence': '0', 'ops': '0.0', 'distance': '0.0', 'all_relations': '', 'chunk': '0', '__trained__': 'CADENCE DESIGN SYSTEMS INC', '__distance_function__': 'levenshtein', '__relation_name__': 'name', 'entity': 'CADENCE DESIGN SYSTEMS INC', 'relation': 'name'}, embeddings=[]), Row(annotatorType='labeled_dependency', begin=0, end=25, result='SERVICES-PREPACKAGED SOFTWARE [7372]', metadata={'sentence': '0', 'ops': '0.0', 'distance': '0.0', 'all_relations': '', 'chunk': '0', '__trained__': 'CADENCE DESIGN SYSTEMS INC', '__distance_function__': 'levenshtein', 

In [21]:
json_dict = dict()
for n in r[0]['mappings']:
    json_dict[n.metadata['relation']] = str(n.result)

In [22]:
import json
print(json.dumps(json_dict, indent=4, sort_keys=True))

{
    "business_city": "SAN JOSE",
    "business_phone": "4089431234",
    "business_state": "CA",
    "business_street": "2655 SEELY AVENUE BLDG 5",
    "business_zip": "95134",
    "company_id": "813672",
    "date": "2017-02-10",
    "fiscal_year_end": "1228",
    "former_name": "ECAD INC /DE/",
    "former_name_date": "19880609",
    "irs_number": "770148231",
    "name": "CADENCE DESIGN SYSTEMS INC",
    "sic": "SERVICES-PREPACKAGED SOFTWARE [7372]",
    "sic_code": "7372",
    "state_incorporation": "DE",
    "state_location": "CA"
}


Yes, here it is. We get additional information about `CADENCE DESIGN SYSTEMS INC` using only company name.